<center>
<img src="https://laelgelcpublic.s3.sa-east-1.amazonaws.com/lael_50_years_narrow_white.png.no_years.400px_96dpi.png" width="300" alt="LAEL 50 years logo">
<h3>APPLIED LINGUISTICS GRADUATE PROGRAMME (LAEL)</h3>
</center>
<hr>

# Corpus Linguistics - Study 1 - Mariana

## Prerequisites

Make sure the prerequisites in [CL_LMDA_prerequisites](https://github.com/laelgelc/laelgelc/blob/main/CL_LMDA_prerequisites.ipynb) are satisfied.

## Dataset

Please download the following dataset (Right-click on the link and choose `Save link as` to download the corresponding file):
- [mari20192020.tsv](https://laelgelcawsemrmariana.s3.sa-east-1.amazonaws.com/mari20192020.tsv)

## Importing the required libraries

In [1]:
import pandas as pd
import demoji
import re
import os
from collections import Counter

## Data wrangling

### Importing the tweet raw data into a dataframe

In [2]:
df_tweets_raw_data = pd.read_csv('mari20192020.tsv', sep='\t')

In [3]:
df_tweets_raw_data.head(5)

,created_at,author_id,username,tweet_url,text
0,2019-01-13 16:02:20+00:00,1057374436624609281,TaconThiago,https://twitter.com/TaconThiago/status/1084480...,RT @RenovaMidia: #URGENTE\n\nServiço de Inteli...
1,2019-01-13 16:02:20+00:00,1057374436624609281,TaconThiago,https://twitter.com/TaconThiago/status/1084480...,RT @RenovaMidia: #URGENTE\n\nServiço de Inteli...
2,2019-01-13 16:00:40+00:00,1076139993599541248,Brunobr18373270,https://twitter.com/Brunobr18373270/status/108...,RT @RenovaMidia: #URGENTE\n\nServiço de Inteli...
3,2019-01-13 16:00:40+00:00,1076139993599541248,Brunobr18373270,https://twitter.com/Brunobr18373270/status/108...,RT @RenovaMidia: #URGENTE\n\nServiço de Inteli...
4,2019-01-13 15:57:54+00:00,1052964117181583361,Pedrodon17,https://twitter.com/Pedrodon17/status/10844796...,RT @RenovaMidia: #URGENTE\n\nServiço de Inteli...


In [4]:
df_tweets_raw_data.shape

(3199, 5)

#### Inspecting a few tweets

In [5]:
inspected_row = 0
print('username:' + df_tweets_raw_data.loc[inspected_row, 'username'])
print('text:' + df_tweets_raw_data.loc[inspected_row, 'text'])
print('tweet_url:' + df_tweets_raw_data.loc[inspected_row, 'tweet_url'])

username:TaconThiago
text:RT @RenovaMidia: #URGENTE

Serviço de Inteligência da ditadura de Nicolás Maduro prendeu o presidente interino da #Venezuela. 

O paradeiro…
tweet_url:https://twitter.com/TaconThiago/status/1084480777365139458


### Inspecting the dataset and eliminating malformed data

#### Checking if data types are consistent

In [6]:
df_tweets_raw_data.dtypes

created_at    object
author_id     object
username      object
tweet_url     object
text          object
dtype: object

#### Identifying rows that are empty in column `text`

In [7]:
print(df_tweets_raw_data['text'].isnull().sum())

0


In [8]:
df_tweets_raw_data[df_tweets_raw_data['text'].isnull()]

,created_at,author_id,username,tweet_url,text


#### Dropping the rows that are empty in the column `text`

In [9]:
# Drop the rows whose column 'text' is NaN
df_tweets_raw_data = df_tweets_raw_data.dropna(subset=['text'])

# Reset the index
df_tweets_raw_data = df_tweets_raw_data.reset_index(drop=True)

In [10]:
print(df_tweets_raw_data['text'].isnull().sum())

0


#### Removing specific Unicode characters

The dataset may need to be cleaned of invisible Unicode characters.

##### Detecting `U+2066` and `U+2069` characters

- [U+2066](https://www.compart.com/en/unicode/U+2066)
- [U+2069](https://www.compart.com/en/unicode/U+2069)

Please refer to:
- [Python RegEx](https://www.w3schools.com/python/python_regex.asp)
- [regex101](https://regex101.com/)
- [RegExr](https://regexr.com/)

In [11]:
# Defining a function to detect specific Unicode characters
def extract_unicode_characters(df, column_name):
    unicode_chars = Counter()  # Initialize a Counter to store Unicode character counts

    for value in df[column_name]:
        if isinstance(value, str):
            # Use RegEx to find non-ASCII characters (Unicode)
#            non_ascii_chars = re.findall(r'[^\x00-\x7F]+', value)
            # Use RegEx to find specific Unicode characters - adjust the expression accordingly
            specific_unicode_chars = re.findall(r'[\u2066\u2069]', value)
            unicode_chars.update(specific_unicode_chars)

    return unicode_chars

# Inspect the dataframe for specific Unicode characters
unicode_counts = extract_unicode_characters(df_tweets_raw_data, 'text')

# Print the results
for char, count in unicode_counts.items():
    print(f'Character {char}: Count = {count}')

Character ⁦: Count = 6
Character ⁩: Count = 6


##### Removing `U+2066` and `U+2069` characters

In [12]:
# Defining a function to remove specific Unicode characters
def remove_specific_unicode(input_line):
    # Using RegEx to replace specific Unicode characters - adjust the expression accordingly
    cleaned_line = re.sub(r'[\u2066\u2069]', '', input_line)
    return cleaned_line

# Removing specific Unicode characters
df_tweets_raw_data['text'] = df_tweets_raw_data['text'].apply(remove_specific_unicode)

#### Replacing the `LF` character by a space

Some tweets, especially the retweeted ones, contain multiple lines of text.

In [13]:
# Defining a function to replace the `LF` character by a space
def remove_cr_lf(input_line):
    # Using RegEx to replace LF by a space
    cleaned_line = re.sub(r'\n', ' ', input_line)
    return cleaned_line

# Applying the function to the 'text' column in your DataFrame
df_tweets_raw_data['text'] = df_tweets_raw_data['text'].apply(remove_cr_lf)

In [14]:
df_tweets_raw_data

,created_at,author_id,username,tweet_url,text
0,2019-01-13 16:02:20+00:00,1057374436624609281,TaconThiago,https://twitter.com/TaconThiago/status/1084480...,RT @RenovaMidia: #URGENTE Serviço de Inteligê...
1,2019-01-13 16:02:20+00:00,1057374436624609281,TaconThiago,https://twitter.com/TaconThiago/status/1084480...,RT @RenovaMidia: #URGENTE Serviço de Inteligê...
2,2019-01-13 16:00:40+00:00,1076139993599541248,Brunobr18373270,https://twitter.com/Brunobr18373270/status/108...,RT @RenovaMidia: #URGENTE Serviço de Inteligê...
3,2019-01-13 16:00:40+00:00,1076139993599541248,Brunobr18373270,https://twitter.com/Brunobr18373270/status/108...,RT @RenovaMidia: #URGENTE Serviço de Inteligê...
4,2019-01-13 15:57:54+00:00,1052964117181583361,Pedrodon17,https://twitter.com/Pedrodon17/status/10844796...,RT @RenovaMidia: #URGENTE Serviço de Inteligê...
...,...,...,...,...,...
3194,2020-12-09 23:26:16+00:00,1272520063619391488,Elcy48639336,https://twitter.com/Elcy48639336/status/133681...,RT @PlenoNews: OEA não reconhece eleições na V...
3195,2020-12-18 21:01:51+00:00,138611877,eduardojo9,https://twitter.com/eduardojo9/status/13400395...,#Venezuela Territorio de Paz 🗣️ ¡𝐓𝐨𝐦𝐚 𝐍𝐨𝐭𝐚!✍️...
3196,2020-12-13 19:24:16+00:00,2948321374,lemp2010,https://twitter.com/lemp2010/status/1338203101...,Atentos #Venezuela #SanFelix @andresscott
3197,2020-12-06 20:59:21+00:00,1096138652214796288,Keila19860,https://twitter.com/Keila19860/status/13356903...,"RT @lcoutinho: #2 A mesma empresa que colhe, p..."


### Dropping duplicates

#### Retweets

Retweets bear the RegEx pattern `'\bRT @\w+\s*:'gm` or `'\bRT @\w+\s':"gm` at the beginning of the column `text`. They should be often dropped because they are duplicates of the original tweets. In the case of this study, though, the dataset may not include the original tweets because it is a 1% random sample. Therefore, only the first occurrence of the retweet is being kept.

In [15]:
# Create a new column 'no_retweet' containing the contents of the column 'text' without any preceding 'RT @mentions:'
df_tweets_raw_data['no_retweet'] = df_tweets_raw_data['text'].str.replace(r'\bRT @\w+\s*:|\brt @\w+\s*:', '', regex=True)

# Drop duplicate rows except the first occurrence based on 'no_mention'
df_tweets_raw_data.drop_duplicates(subset='no_retweet', keep='first', inplace=True)
df_tweets_raw_data = df_tweets_raw_data.reset_index(drop=True)
df_tweets_raw_data.shape

(1697, 6)

#### Duplicate tweets

The dataset was build in a way that if a certain tweet had more than one photo, one copy of the tweet was included per unique photo. Since we are concerned with analysing just the text, those duplicates should be removed. Tweets that bear the same 'tweet_url' are duplicates - we are going to keep only the first.

In [16]:
df_tweets_raw_data.drop_duplicates(subset='tweet_url', keep='first', inplace=True)
df_tweets_raw_data = df_tweets_raw_data.reset_index(drop=True)
df_tweets_raw_data.shape

(1697, 6)

#### @mentioned tweets

A few users @mention copies of tweets towards other specific users creating multiple copies of the same tweet - those duplicates should be removed.

In [17]:
# Create a new column 'no_mention' containing the contents of the column 'text' without any preceding @mentions
df_tweets_raw_data['no_mention'] = df_tweets_raw_data['text'].str.replace(r'@\w+\s*', '', regex=True)

# Drop duplicate rows except the first occurrence based on 'no_mention'
df_tweets_raw_data.drop_duplicates(subset='no_mention', keep='first', inplace=True)
df_tweets_raw_data = df_tweets_raw_data.reset_index(drop=True)
df_tweets_raw_data.shape

(1697, 7)

## Sampling the raw data according to filtering expressions

In [18]:
# Defining the filtering expressions
#filter_words = ['arma', 'pátria', 'ladrão', 'cristão', 'comunista', 'família', 'liberdade', 'conservador', 'deus']
filter_words = ['venezuela']

# Creating a boolean mask for filtering
mask = df_tweets_raw_data['text'].str.contains('|'.join(filter_words), case=False)

# Applying the mask to create 'df_tweets_filtered'
df_tweets_filtered = df_tweets_raw_data[mask]
df_tweets_filtered = df_tweets_filtered.reset_index(drop=True)

In [19]:
df_tweets_filtered

,created_at,author_id,username,tweet_url,text,no_retweet,no_mention
0,2019-01-13 16:02:20+00:00,1057374436624609281,TaconThiago,https://twitter.com/TaconThiago/status/1084480...,RT @RenovaMidia: #URGENTE Serviço de Inteligê...,#URGENTE Serviço de Inteligência da ditadura...,RT : #URGENTE Serviço de Inteligência da dita...
1,2019-01-13 16:13:37+00:00,2561336983,Rebuscardesign,https://twitter.com/Rebuscardesign/status/1084...,RT @RenovaMidia: Momento em que o presidente i...,Momento em que o presidente interino da #Vene...,RT : Momento em que o presidente interino da #...
2,2019-01-13 16:33:21+00:00,27660599,wilmenbr,https://twitter.com/wilmenbr/status/1084488582...,RT @forumvoto: #Venezuela | Presidente da asse...,#Venezuela | Presidente da assembleia naciona...,RT : #Venezuela | Presidente da assembleia nac...
3,2019-01-14 13:04:27+00:00,841372125432827905,EmersonCAmorim,https://twitter.com/EmersonCAmorim/status/1084...,RT @RenovaMidia: O colapso social gerado pela ...,O colapso social gerado pela ditadura de Nico...,RT : O colapso social gerado pela ditadura de ...
4,2019-01-15 12:18:42+00:00,3156620115,DcsbeleDaniele,https://twitter.com/DcsbeleDaniele/status/1085...,"RT @RenovaMidia: Com este aumento, o salário m...","Com este aumento, o salário mínimo na #Venezu...","RT : Com este aumento, o salário mínimo na #Ve..."
...,...,...,...,...,...,...,...
1674,2020-12-05 08:42:17+00:00,4295131347,e_GNOP,https://twitter.com/e_GNOP/status/133514243817...,Venezuela: Nicolás Maduro renunciará se perder...,Venezuela: Nicolás Maduro renunciará se perder...,Venezuela: Nicolás Maduro renunciará se perder...
1675,2020-12-09 23:26:16+00:00,1272520063619391488,Elcy48639336,https://twitter.com/Elcy48639336/status/133681...,RT @PlenoNews: OEA não reconhece eleições na V...,OEA não reconhece eleições na Venezuela e acu...,RT : OEA não reconhece eleições na Venezuela e...
1676,2020-12-13 19:24:16+00:00,2948321374,lemp2010,https://twitter.com/lemp2010/status/1338203101...,Atentos #Venezuela #SanFelix @andresscott,Atentos #Venezuela #SanFelix @andresscott,Atentos #Venezuela #SanFelix
1677,2020-12-06 20:59:21+00:00,1096138652214796288,Keila19860,https://twitter.com/Keila19860/status/13356903...,"RT @lcoutinho: #2 A mesma empresa que colhe, p...","#2 A mesma empresa que colhe, processa e apur...","RT : #2 A mesma empresa que colhe, processa e ..."


### Exporting the filtered data into a file for inspection

In [20]:
df_tweets_filtered.to_csv('tweets_emojified.tsv', sep='\t', index=False)

## Replacing emojis

### Demojifying the column `text`

In [21]:
# Defining a function to demojify a string
def demojify_line(input_line):
    demojified_line = demoji.replace_with_desc(input_line, sep='<em>')
    return demojified_line

df_tweets_filtered['text'] = df_tweets_filtered['text'].apply(demojify_line)

#### Exporting the filtered data into a file for inspection

In [22]:
df_tweets_filtered.to_csv('tweets_demojified1.tsv', sep='\t', index=False)

### Separating the demojified strings with spaces

In [23]:
# Defining a function to separate the demojified strings with spaces
def preprocess_line(input_line):
    # Add a space before the first delimiter '<em>', if it is not already preceded by one
    preprocessed_line = re.sub(r'(?<! )<em>', ' <em>', input_line)
    # Add a space after the first delimiter '<em>', if it is not already followed by one
    preprocessed_line = re.sub(r'<em>(?! )', '<em> ', preprocessed_line)
    return preprocessed_line

# Separating the demojified strings with spaces
df_tweets_filtered['text'] = df_tweets_filtered['text'].apply(preprocess_line)

#### Exporting the filtered data into a file for inspection

In [24]:
df_tweets_filtered.to_csv('tweets_demojified2.tsv', sep='\t', index=False)

### Formatting the demojified strings

In [25]:
# Defining a function to format the demojified string
def format_demojified_string(input_line):
    # Defining a function to format the demojified string using RegEx
    def process_demojified_string(s):
            # Lowercase the string
            s = s.lower()
            # Replace spaces and colons followed by a space with underscores
            s = re.sub(r'(: )| ', '_', s)
            # Add the appropriate prefixes and suffixes
            s = f'EMOJI{s}e'
            return s

    # Use RegEx to find and process each demojified string
    processed_line = re.sub(r'<em>(.*?)<em>', lambda match: process_demojified_string(match.group(1)), input_line)
    return processed_line

# Formatting the demojified strings
df_tweets_filtered['text'] = df_tweets_filtered['text'].apply(format_demojified_string)

### Replacing the `pipe` character by the `-` character in the `text` column

Further on, a few columns of the dataframe are going to be exported into the file `tweets.txt` whose columns need to be delimited by the `pipe` character. Therefore, it is recommended that any occurrences of the `pipe` character in the `text` column are replaced by another character.

In [26]:
# Defining a function to replace the 'pipe' character by the '-' character
def replace_pipe_with_hyphen(input_string):
    modified_string = re.sub(r'\|', '-', input_string)
    return modified_string

# Replacing the 'pipe' character by the '-' character
df_tweets_filtered['text'] = df_tweets_filtered['text'].apply(replace_pipe_with_hyphen)


#### Exporting the filtered data into a file for inspection

In [27]:
df_tweets_filtered.to_csv('tweets_demojified3.tsv', sep='\t', index=False)

## Tokenising

Please refer to [What is tokenization in NLP?](https://www.analyticsvidhya.com/blog/2020/05/what-is-tokenization-nlp/).

In [28]:
# Defining a function to tokenise a string
def tokenise_string(input_line):
    # Replace URLs with placeholders
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+\b'
    placeholder = '<URL>'  # Choose a unique placeholder
    urls = re.findall(url_pattern, input_line)
    tokenised_line = re.sub(url_pattern, placeholder, input_line)  # Replace URLs with placeholders
    
    # Replace curly quotes with straight ones
    tokenised_line = tokenised_line.replace('“', '"').replace('”', '"').replace("‘", "'").replace("’", "'")
    # Separate common punctuation marks with spaces
    tokenised_line = re.sub(r'([.\!?,"\'/()])', r' \1 ', tokenised_line)
    # Add a space before '#'
    tokenised_line = re.sub(r'(?<!\s)#', r' #', tokenised_line)  # Add a space before '#' if it is not already preceded by one
    # Reduce extra spaces by a single space
    tokenised_line = re.sub(r'\s+', ' ', tokenised_line)
    
    # Replace the placeholders with the respective URLs
    for url in urls:
        tokenised_line = tokenised_line.replace(placeholder, url, 1)
    
    return tokenised_line

# Tokenising the strings
df_tweets_filtered['text'] = df_tweets_filtered['text'].apply(tokenise_string)

## Creating the files `file_index.txt` and `tweets.txt`

### Creating column `text_id`

In [29]:
df_tweets_filtered['text_id'] = 't' + df_tweets_filtered.index.astype(str).str.zfill(6)

### Creating column `conversation`

In [30]:
df_tweets_filtered['conversation'] = 'v:' + df_tweets_filtered['author_id'].str.replace('id_', '')

### Creating column `date`

In [31]:
# Convert 'created_at' to datetime format
df_tweets_filtered['created_at'] = pd.to_datetime(df_tweets_filtered['created_at'])

# Extract the date part (without time) into a new column 'date'
df_tweets_filtered['date'] = df_tweets_filtered['created_at'].dt.date

# Add the prefix 'd:' to the 'date' values
df_tweets_filtered['date'] = 'd:' + df_tweets_filtered['date'].astype(str)

### Creating column `text_url`

In [32]:
df_tweets_filtered['text_url'] = 'url:' + df_tweets_filtered['tweet_url']

### Creating column `user`

In [33]:
df_tweets_filtered['user'] = 'u:' + df_tweets_filtered['username']

### Creating column `content`

In [34]:
df_tweets_filtered['content'] = 'c:' + df_tweets_filtered['text']

### Reordering the created columns

Please refer to:
- [Python - List Comprehension 1](https://www.w3schools.com/python/python_lists_comprehension.asp)
- [Python - List Comprehension 2](https://treyhunner.com/2015/12/python-list-comprehensions-now-in-color/)

In [35]:
# Reorder the columns (we use list comprehension to create a list of all columns except 'text_id', 'variable', 'date' and 'text_url')
df_tweets_filtered = df_tweets_filtered[['text_id', 'conversation', 'date', 'text_url', 'user', 'content'] + [col for col in df_tweets_filtered.columns if col not in ['text_id', 'conversation', 'date', 'text_url', 'user', 'content']]]

In [36]:
df_tweets_filtered

,text_id,conversation,date,text_url,user,content,created_at,author_id,username,tweet_url,text,no_retweet,no_mention
0,t000000,v:1057374436624609281,d:2019-01-13,url:https://twitter.com/TaconThiago/status/108...,u:TaconThiago,c:RT @RenovaMidia: #URGENTE Serviço de Intelig...,2019-01-13 16:02:20+00:00,1057374436624609281,TaconThiago,https://twitter.com/TaconThiago/status/1084480...,RT @RenovaMidia: #URGENTE Serviço de Inteligên...,#URGENTE Serviço de Inteligência da ditadura...,RT : #URGENTE Serviço de Inteligência da dita...
1,t000001,v:2561336983,d:2019-01-13,url:https://twitter.com/Rebuscardesign/status/...,u:Rebuscardesign,c:RT @RenovaMidia: Momento em que o presidente...,2019-01-13 16:13:37+00:00,2561336983,Rebuscardesign,https://twitter.com/Rebuscardesign/status/1084...,RT @RenovaMidia: Momento em que o presidente i...,Momento em que o presidente interino da #Vene...,RT : Momento em que o presidente interino da #...
2,t000002,v:27660599,d:2019-01-13,url:https://twitter.com/wilmenbr/status/108448...,u:wilmenbr,c:RT @forumvoto: #Venezuela - Presidente da as...,2019-01-13 16:33:21+00:00,27660599,wilmenbr,https://twitter.com/wilmenbr/status/1084488582...,RT @forumvoto: #Venezuela - Presidente da asse...,#Venezuela | Presidente da assembleia naciona...,RT : #Venezuela | Presidente da assembleia nac...
3,t000003,v:841372125432827905,d:2019-01-14,url:https://twitter.com/EmersonCAmorim/status/...,u:EmersonCAmorim,c:RT @RenovaMidia: O colapso social gerado pel...,2019-01-14 13:04:27+00:00,841372125432827905,EmersonCAmorim,https://twitter.com/EmersonCAmorim/status/1084...,RT @RenovaMidia: O colapso social gerado pela ...,O colapso social gerado pela ditadura de Nico...,RT : O colapso social gerado pela ditadura de ...
4,t000004,v:3156620115,d:2019-01-15,url:https://twitter.com/DcsbeleDaniele/status/...,u:DcsbeleDaniele,"c:RT @RenovaMidia: Com este aumento , o salári...",2019-01-15 12:18:42+00:00,3156620115,DcsbeleDaniele,https://twitter.com/DcsbeleDaniele/status/1085...,"RT @RenovaMidia: Com este aumento , o salário ...","Com este aumento, o salário mínimo na #Venezu...","RT : Com este aumento, o salário mínimo na #Ve..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1674,t001674,v:4295131347,d:2020-12-05,url:https://twitter.com/e_GNOP/status/13351424...,u:e_GNOP,c:Venezuela: Nicolás Maduro renunciará se perd...,2020-12-05 08:42:17+00:00,4295131347,e_GNOP,https://twitter.com/e_GNOP/status/133514243817...,Venezuela: Nicolás Maduro renunciará se perder...,Venezuela: Nicolás Maduro renunciará se perder...,Venezuela: Nicolás Maduro renunciará se perder...
1675,t001675,v:1272520063619391488,d:2020-12-09,url:https://twitter.com/Elcy48639336/status/13...,u:Elcy48639336,c:RT @PlenoNews: OEA não reconhece eleições na...,2020-12-09 23:26:16+00:00,1272520063619391488,Elcy48639336,https://twitter.com/Elcy48639336/status/133681...,RT @PlenoNews: OEA não reconhece eleições na V...,OEA não reconhece eleições na Venezuela e acu...,RT : OEA não reconhece eleições na Venezuela e...
1676,t001676,v:2948321374,d:2020-12-13,url:https://twitter.com/lemp2010/status/133820...,u:lemp2010,c:Atentos #Venezuela #SanFelix @andresscott,2020-12-13 19:24:16+00:00,2948321374,lemp2010,https://twitter.com/lemp2010/status/1338203101...,Atentos #Venezuela #SanFelix @andresscott,Atentos #Venezuela #SanFelix @andresscott,Atentos #Venezuela #SanFelix
1677,t001677,v:1096138652214796288,d:2020-12-06,url:https://twitter.com/Keila19860/status/1335...,u:Keila19860,c:RT @lcoutinho: #2 A mesma empresa que colhe ...,2020-12-06 20:59:21+00:00,1096138652214796288,Keila19860,https://twitter.com/Keila19860/status/13356903...,"RT @lcoutinho: #2 A mesma empresa que colhe , ...","#2 A mesma empresa que colhe, processa e apur...","RT : #2 A mesma empresa que colhe, processa e ..."


### Creating the file `file_index.txt`

In [37]:
df_tweets_filtered[['text_id', 'conversation', 'date', 'text_url']].to_csv('file_index.txt', sep=' ', index=False, header=False, encoding='utf-8', lineterminator='\n')

### Creating the file `tweets.txt`

In [38]:
folder = 'tweets'
try:
    os.mkdir(folder)
    print(f'Folder {folder} created!')
except FileExistsError:
    print(f'Folder {folder} already exists')

Folder tweets created!


Note: The parameters `doublequote=False` and `escapechar=' '` are required to avoid that the column content is doublequoted with '"' in sentences that use characters that need to be escaped such as double quote '"' itself - this causes a malformed response from TreeTagger.

In [39]:
df_tweets_filtered[['text_id', 'conversation', 'date', 'user', 'content']].to_csv(f'{folder}/tweets.txt', sep='|', index=False, header=False, encoding='utf-8', lineterminator='\n', doublequote=False, escapechar=' ')

## Tagging with TreeTagger

- On Visual Studio Code (VS Code), open the folder where your project is located with `Open Folder...`
- Open a WSL Ubuntu Terminal on VS Code
- **Important**: Activate the `my_env` Python environment by executing `source "$HOME"/my_env/bin/activate`
- Proceed as indicated

Note: You have to download and open this Jupyter Notebook on JupyterLab (provided as part of Anaconda Distribution) to visualise the procedure

Purpose: Annotate the texts in `tweets/tweets.txt` with part-of-speech and lemma information.
- Input
    - `file_index.txt`
    - `tweets/tweets.txt`
- Output
    - `tweets/tagged.txt`

## Processing `tokenstypes`

Purpose: Capture the content tokens (specific occurrences of words) and the content types (general concept of words) from `tweets/tagged.txt`.
- Input
    - `file_index.txt`
    - `tweets/tagged.txt`
- Output
    - `tweets/tokens.txt`
    - `tweets/types.txt`

## Processing `toplemmas`

Purpose: Determine the 1.000 top lemmas. **Important**: This process requires manual inspection. Non-meaningful lemmas should be excluded by updating `stoplist.sed` and reiterating the processing.
- Input
    - `tweets/types.txt`
    - `stoplist.sed`: List of rules that allows the exclusion of a certain lemmas
- Output
    - `selectedwords` = `var_index.txt`

## Processing `sas`

Purpose: Prepare input data for processing in SAS.
- Input
    - `tweets/types.txt`
    - `selectedwords`
    - `file_index.txt`
- Output
    - `columns`
    - `sas/data.txt`
    - `sas/dates.txt`
    - `sas/wcount.txt`

## Processing `datamatrix`

Purpose: Prepares input data for calculating the correlation matrix.
- Input
    - `file_index.txt`
    - `columns`
    - `selectedwords`
- Output
    - `file_ids.txt`
    - `data.csv`

## Processing `correlationmatrix`

Purpose: Calculates the correlation matrix.
- Input
    - `data.csv`
- Output
    - `correlation`

## Processing `formats`

Purpose: Prepare input data for processing in SAS.
- Input
    - `data.csv`
    - `selectedwords`
- Output
    - `sas/corr.txt`
    - `sas/word_labels_format.sas`

## Processing the statistical procedures on SAS

- Log in to your [SAS OnDemand for Academics](https://welcome.oda.sas.com/) account
- Proceed as indicated in this [video tutorial](https://youtu.be/I3u9zD3jyOA?si=68uIKVc2iusGG2KY)

## Processing `examples`

Purpose: Extract examples for analysis.
- Input
    - `sas/output_"$project"/loadtable.html`
    - `sas/output_"$project"/"$project"_scores.tsv`
    - `sas/output_"$project"/"$project"_scores_only.tsv`
- Output
    - `examples/factors`
    - `example files`

## Results

Right-click on the link and choose `Save link as` to download the corresponding file.

- [CL_St1_Mariana_Results.zip](https://laelgelcawsemrmariana.s3.sa-east-1.amazonaws.com/CL_St1_Mariana_Results.zip)